In [1]:
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.12:0.14.0 pyspark-shell'

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
df = spark.read.option("delimiter", "\t").csv('hdfs://hw2-m/user/root/p1t2_small')

:: loading settings :: url = jar:file:/usr/lib/spark/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
com.databricks#spark-xml_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-4187b8f0-a486-4de7-85c3-6fdd5c3e2f9a;1.0
	confs: [default]
	found com.databricks#spark-xml_2.12;0.14.0 in central
	found commons-io#commons-io;2.8.0 in central
	found org.glassfish.jaxb#txw2;2.3.4 in central
	found org.apache.ws.xmlschema#xmlschema-core;2.2.5 in central
:: resolution report :: resolve 278ms :: artifacts dl 7ms
	:: modules in use:
	com.databricks#spark-xml_2.12;0.14.0 from central in [default]
	commons-io#commons-io;2.8.0 from central in [default]
	org.apache.ws.xmlschema#xmlschema-core;2.2.5 from central in [default]
	org.glassfish.jaxb#txw2;2.3.4 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|

In [3]:
df = df.filter("_c1 is not null")
df.count()

4413753

In [4]:
from pyspark.sql.functions import lit

df_count = df.groupBy("_c0").count()
newdf = df_count.withColumn("rank", lit(1))
newdf = newdf.rdd.map(lambda x: (x[0], x[1], x[2], x[2]/x[1])).toDF(['article', 'count', 'rank', 'contribution'])

In [5]:
newdf.show()

22/04/22 21:43:52 WARN org.apache.spark.deploy.yarn.YarnAllocator: Container from a bad node: container_1650653302475_0004_01_000004 on host: hw2-w-0.c.csee4121-339916.internal. Exit status: 134. Diagnostics: [2022-04-22 21:43:52.769]Exception from container-launch.
Container id: container_1650653302475_0004_01_000004
Exit code: 134

[2022-04-22 21:43:52.772]Container exited with a non-zero exit code 134. Error file: prelaunch.err.
Last 4096 bytes of prelaunch.err :
/bin/bash: line 1:  4670 Aborted                 /usr/lib/jvm/temurin-8-jdk-amd64/bin/java -server -Xmx5739m -Djava.io.tmpdir=/hadoop/yarn/nm-local-dir/usercache/root/appcache/application_1650653302475_0004/container_1650653302475_0004_01_000004/tmp '-Dspark.driver.port=44927' '-Dspark.ui.port=0' '-Dspark.rpc.message.maxSize=512' -Dspark.yarn.app.container.log.dir=/var/log/hadoop-yarn/userlogs/application_1650653302475_0004/container_1650653302475_0004_01_000004 -XX:OnOutOfMemoryError='kill %p' org.apache.spark.executor.Yar

+--------------------+-----+----+--------------------+
|             article|count|rank|        contribution|
+--------------------+-----+----+--------------------+
|hypothetical type...|  406|   1|0.002463054187192...|
|guantanamo bay na...|  160|   1|             0.00625|
|     jerzy grotowski|   56|   1|0.017857142857142856|
|georg wilhelm fri...|  527|   1|0.001897533206831...|
|johns hopkins uni...|  391|   1|0.002557544757033...|
|johann heinrich a...|   24|   1|0.041666666666666664|
|      greek language|  315|   1|0.003174603174603...|
|geography of moldova|  148|   1|0.006756756756756757|
|                 fog|  159|   1|0.006289308176100629|
|            guernsey|  311|   1|0.003215434083601286|
|       gary, indiana|  230|   1|0.004347826086956522|
|       georges perec|  111|   1|0.009009009009009009|
|      frederick abel|   69|   1|0.014492753623188406|
|       james madison|  296|   1|0.003378378378378...|
|             hydrate|   53|   1|0.018867924528301886|
|james mad

In [ ]:
for _ in range(10):
    newdf = df.join(newdf, df._c0 == newdf.article, 'inner')
    newdf = newdf.groupBy(['_c1']).sum('contribution').rdd.map(lambda x: (x[0], 0.15 + 0.85 * x[1])).toDF(['article', 'rank'])
    newdf = df_count.join(newdf, df._c0 == newdf.article, 'left')
    newdf = newdf.na.fill(0.15)
    newdf = newdf.rdd.map(lambda x:(x[0], x[1], x[3], x[3]/x[1])).toDF(['article', 'count', 'rank', 'contribution'])

22/04/22 21:44:09 WARN org.apache.spark.deploy.yarn.YarnAllocator: Container from a bad node: container_1650653302475_0004_01_000002 on host: hw2-w-0.c.csee4121-339916.internal. Exit status: 134. Diagnostics: [2022-04-22 21:44:09.330]Exception from container-launch.
Container id: container_1650653302475_0004_01_000002
Exit code: 134

[2022-04-22 21:44:09.331]Container exited with a non-zero exit code 134. Error file: prelaunch.err.
Last 4096 bytes of prelaunch.err :
/bin/bash: line 1:  4616 Aborted                 /usr/lib/jvm/temurin-8-jdk-amd64/bin/java -server -Xmx5739m -Djava.io.tmpdir=/hadoop/yarn/nm-local-dir/usercache/root/appcache/application_1650653302475_0004/container_1650653302475_0004_01_000002/tmp '-Dspark.driver.port=44927' '-Dspark.ui.port=0' '-Dspark.rpc.message.maxSize=512' -Dspark.yarn.app.container.log.dir=/var/log/hadoop-yarn/userlogs/application_1650653302475_0004/container_1650653302475_0004_01_000002 -XX:OnOutOfMemoryError='kill %p' org.apache.spark.executor.Yar

22/04/22 21:45:50 WARN org.apache.spark.deploy.yarn.YarnAllocator: Container from a bad node: container_1650653302475_0004_01_000005 on host: hw2-w-0.c.csee4121-339916.internal. Exit status: 134. Diagnostics: [2022-04-22 21:45:50.654]Exception from container-launch.
Container id: container_1650653302475_0004_01_000005
Exit code: 134

[2022-04-22 21:45:50.655]Container exited with a non-zero exit code 134. Error file: prelaunch.err.
Last 4096 bytes of prelaunch.err :
/bin/bash: line 1:  4768 Aborted                 /usr/lib/jvm/temurin-8-jdk-amd64/bin/java -server -Xmx5739m -Djava.io.tmpdir=/hadoop/yarn/nm-local-dir/usercache/root/appcache/application_1650653302475_0004/container_1650653302475_0004_01_000005/tmp '-Dspark.driver.port=44927' '-Dspark.ui.port=0' '-Dspark.rpc.message.maxSize=512' -Dspark.yarn.app.container.log.dir=/var/log/hadoop-yarn/userlogs/application_1650653302475_0004/container_1650653302475_0004_01_000005 -XX:OnOutOfMemoryError='kill %p' org.apache.spark.executor.Yar

In [ ]:
newdf = newdf.select('article', 'rank').orderBy('article', 'rank')
newdf.show()

In [9]:
newdf.repartition(10).write.option("delimiter", "\t").csv('p1t3_small')

In [10]:
newdf.limit(5).toPandas().to_csv('/home/jz3313/hw2/p1t3/p1t3.csv', sep='\t', index=False)